In [ ]:
# Fancaps Direct URL Downloader
# By citronlegacy:
# https://github.com/citronlegacy/fancaps-direct-url-downloader

import os
import time
import concurrent.futures
import urllib.request
import zipfile

# Paste your URLs here (one URL per line)
urls_text = """
https://cdni.fancaps.net/file/fancaps-movieimages/4839637.jpg
https://cdni.fancaps.net/file/fancaps-movieimages/4839642.jpg
https://cdni.fancaps.net/file/fancaps-movieimages/4839650.jpg
"""

# -------------------------
# User Configuration
# -------------------------

# Define download directory and zip option
download_directory = "test"  # @param {type:"string"}
zip_folder = True  # @param {type:"boolean"}

USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

# Output folder
OUTPUT_FOLDER = download_directory
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Pause configuration
PAUSE_EVERY_N = 3

# -------------------------
# Parse URLs
# -------------------------

image_urls = [line.strip() for line in urls_text.strip().split('\n') if line.strip()]

# -------------------------
# HTTP headers
# -------------------------

headers = {
    'User-Agent': USER_AGENT
}

# -------------------------
# Download function
# -------------------------

def download_file(url):
    filename = os.path.basename(url)
    filepath = os.path.join(OUTPUT_FOLDER, filename)
    print(f"Downloading: {url} -> {filepath}")
    try:
        req = urllib.request.Request(url, headers=headers)
        with urllib.request.urlopen(req) as resp, open(filepath, 'wb') as out_file:
            out_file.write(resp.read())
        print(f"✅ Saved: {filename}")
    except Exception as e:
        print(f"❌ Error downloading {url}: {e}")

# -------------------------
# Run downloads with pauses
# -------------------------

def download_all_with_pauses(urls):
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        futures = []
        for i, url in enumerate(urls):
            futures.append(executor.submit(download_file, url))
            if (i + 1) % PAUSE_EVERY_N == 0:
                print(f"\nℹ️  Pausing for 1 second every {PAUSE_EVERY_N} downloads...\n")
                time.sleep(1)
        concurrent.futures.wait(futures)

# -------------------------
# Zip the folder if enabled
# -------------------------

def zip_downloads(folder_path):
    zip_filename = f"{folder_path}.zip"
    print(f"\n📦 Zipping folder to: {zip_filename}")
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)
    print("✅ Zipping complete.")

# -------------------------
# Run everything
# -------------------------

download_all_with_pauses(image_urls)

if zip_folder:
    zip_downloads(OUTPUT_FOLDER)

print("🎉 Done downloading all images.")
